# Reading SOFA Files With Python

SOFA: http://www.sofaconventions.org/

Example file from https://depositonce.tu-berlin.de/handle/11303/245.2, also available at http://sofacoustics.org/data/database/tuburo/.

This is only about *reading* files, *creating* and *writing* to SOFA files is beyond the scope of this page.

## scipy.io.netcdf

scipy.io.netcdf (v0.18) doesn't support NetCDF4.

http://docs.scipy.org/doc/scipy/reference/generated/scipy.io.netcdf.netcdf_file.html

In [1]:
from scipy.io import netcdf_file

In [2]:
try:
    netcdf_file('RIR_AllAbsorbers_ArrayCentre_Emitters1to64.sofa')
except Exception as e:
    print(e)

Error: RIR_AllAbsorbers_ArrayCentre_Emitters1to64.sofa is not a valid NetCDF 3 file


In [3]:
import scipy
scipy.__version__

'0.18.1'

## netcdf4-python

based on Scientific.IO.NetCDF API

Debian package `python3-netcdf4`

http://unidata.github.io/netcdf4-python/

https://github.com/Unidata/netcdf4-python

http://nbviewer.ipython.org/github/Unidata/netcdf4-python/blob/master/examples/reading_netCDF.ipynb

http://nbviewer.ipython.org/github/Unidata/netcdf4-python/blob/master/examples/writing_netCDF.ipynb

In [4]:
import netCDF4

In [5]:
f = netCDF4.Dataset('RIR_AllAbsorbers_ArrayCentre_Emitters1to64.sofa')
f

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: SOFA
    Version: 0.6
    SOFAConventions: GeneralFIRE
    SOFAConventionsVersion: 0.1
    APIName: ARI SOFA API for Matlab/Octave
    APIVersion: 0.4.4
    ApplicationName: Matlab
    ApplicationVersion: R2013a
    AuthorContact: vera.erbes@uni-rostock.de
    Comment: RIR measurements of 64-channel loudspeaker array at University of Rostock.
    DataType: FIRE
    History: Converted from the TU Berlin/University of Rostock format
    License: Creative Commons Attribution-NonCommercial-ShareAlike 4.0
    Organization: University of Rostock, Institute of Communications Engineering
    References: V. Erbes, M. Geier, S. Weinzierl and S. Spors (2015): Database of single-channel and binaural room impulse responses of a 64-channel loudspeaker array. Proc. of the 138th AES Conv., Warsaw, Poland
    RoomType: shoebox
    Origin: Acoustically measured with omnidirectional microphone
    DateC

In [6]:
f.variables

OrderedDict([('ListenerPosition', <class 'netCDF4._netCDF4.Variable'>
              float64 ListenerPosition(I, C)
                  Type: cartesian
                  Units: meter
              unlimited dimensions: 
              current shape = (1, 3)
              filling on, default _FillValue of 9.969209968386869e+36 used),
             ('ReceiverPosition', <class 'netCDF4._netCDF4.Variable'>
              float64 ReceiverPosition(R, C, I)
                  Type: cartesian
                  Units: meter
              unlimited dimensions: 
              current shape = (1, 3, 1)
              filling on, default _FillValue of 9.969209968386869e+36 used),
             ('SourcePosition', <class 'netCDF4._netCDF4.Variable'>
              float64 SourcePosition(I, C)
                  Type: cartesian
                  Units: meter
              unlimited dimensions: 
              current shape = (1, 3)
              filling on, default _FillValue of 9.969209968386869e+36 used),
     

In [7]:
var = f.variables['Data.IR']
var

<class 'netCDF4._netCDF4.Variable'>
float64 Data.IR(M, R, E, N)
unlimited dimensions: 
current shape = (1, 1, 64, 44100)
filling on, default _FillValue of 9.969209968386869e+36 used

In [8]:
data = var[0, 0]
data.shape

(64, 44100)

In [9]:
f.close()

In [10]:
netCDF4.__version__

'1.2.7'

In [11]:
netCDF4.__netcdf4libversion__

'4.4.1.1'

In [12]:
netCDF4.__hdf5libversion__

'1.10.0-patch1'

## Scientific.IO.NetCDF

ScientificPython (not to be confused with SciPy!)

http://dirac.cnrs-orleans.fr/plone/software/scientificpython

https://bitbucket.org/khinsen/scientificpython

http://dirac.cnrs-orleans.fr/ScientificPython/ScientificPythonManual/Scientific.IO.NetCDF.NetCDFFile-class.html

Only for Python 2, no Python 3 support?

Example:

```python
from Scientific.IO.NetCDF import NetCDFFile

f = NetCDFFile('RIR_AllAbsorbers_ArrayCentre_Emitters1to64.sofa')

var = f.variables['Data.IR']

var.typecode()  # 'd'

data = var.getValue()

data.shape  # (1, 1, 64, 44100)
```

## PyTables

Open SOFA file as HDF5 (there are a lot of warnings but it seems to work)

http://www.pytables.org/

In [13]:
import tables

In [14]:
f = tables.open_file('RIR_AllAbsorbers_ArrayCentre_Emitters1to64.sofa')

In [15]:
#f

In [16]:
#f.root

It's impossible to get `Data.IR` by attribute access because sadly it contains a period.

In [17]:
var = f.get_node('/Data.IR')
var

/usr/lib/python3/dist-packages/tables/attributeset.py:298: DataTypeWarning: Unsupported type for attribute 'DIMENSION_LIST' in node 'Data.IR'. Offending HDF5 class: 9
  value = self._g_getattr(self._v_node, name)


/Data.IR (CArray(1, 1, 64, 44100), shuffle, zlib(1)) ''
  atom := Float64Atom(shape=(), dflt=9.969209968386869e+36)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (1, 1, 20, 14596)

`CArray` $\to$ chunked array

In [18]:
var.dtype

dtype('float64')

In [19]:
var.shape

(1, 1, 64, 44100)

In [20]:
data = var[0, 0]
type(data)

numpy.ndarray

In [21]:
f.close()

In [22]:
tables.__version__

'3.3.0'

## h5py

http://www.h5py.org/

http://docs.h5py.org/

In [23]:
import h5py

In [24]:
f = h5py.File('RIR_AllAbsorbers_ArrayCentre_Emitters1to64.sofa')
f

<HDF5 file "RIR_AllAbsorbers_ArrayCentre_Emitters1to64.sofa" (mode r+)>

In [25]:
var = f['Data.IR']
var

<HDF5 dataset "Data.IR": shape (1, 1, 64, 44100), type "<f8">

In [26]:
data = var[0, 0]
type(data)

numpy.ndarray

In [27]:
f.close()

In [28]:
h5py.__version__

'2.6.0'

## h5netcdf

Uses the `h5py` module (see above). The NetCDF4 C library is *not* needed.

https://pypi.python.org/pypi/h5netcdf/

https://github.com/shoyer/h5netcdf

In [29]:
import h5netcdf

In [30]:
f = h5netcdf.File('RIR_AllAbsorbers_ArrayCentre_Emitters1to64.sofa', 'r')

In [31]:
f

<h5netcdf.File 'RIR_AllAbsorbers_ArrayCentre_Emitters1to64.sofa' (mode r)>
Dimensions:
    E: 64
    N: 44100
    I: 1
    R: 1
    M: 1
    S: 0
    C: 3
Groups:
Variables:
    ListenerPosition: ('I', 'C') float64
    SourceUp: ('I', 'C') float64
    EmitterView: ('E', 'C', 'I') float64
    RoomCornerA: ('I', 'C') float64
    SourceView: ('I', 'C') float64
    EmitterUp: ('E', 'C', 'I') float64
    EmitterPosition: ('E', 'C', 'I') float64
    Data.Delay: ('I', 'R', 'E') float64
    Data.IR: ('M', 'R', 'E', 'N') float64
    RoomCornerB: ('I', 'C') float64
    ReceiverPosition: ('R', 'C', 'I') float64
    SourcePosition: ('I', 'C') float64
    Data.SamplingRate: ('I',) float64
Attributes:
    RoomDescription: b'University of Rostock, INT Audio Lab, Room 8202, broadband absorbers on walls and ceiling, mid-frequency reverberation time = 0.32 s'
    ApplicationVersion: b'R2013a'
    DatabaseName: b'Multichannel RIRs University of Rostock'
    SourceDescription: b'64-channel loudspeaker arr

In [32]:
var = f['Data.IR']
var

<h5netcdf.Variable '/Data.IR': dimensions ('M', 'R', 'E', 'N'), shape (1, 1, 64, 44100), dtype float64>
Attributes:

In [33]:
data = var[0, 0]
type(data)

numpy.ndarray

In [34]:
f.close()

In [35]:
h5netcdf.__version__

'0.3.1'